## instalación de librerias

In [1]:
!pip install nltk
!pip install contractions
!pip install inflect
!pip install scikit-plot
!pip install spacy
!pip install stanza
!pip install xgboost

You should consider upgrading via the 'C:\Users\Julian\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Users\Julian\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Users\Julian\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.
You should consider upgrading via the 'C:\Users\Julian\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Users\Julian\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Users\Julian\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Users\Julian\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [2]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Julian\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Julian\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Julian\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [3]:
# Instalación de librerias
import pandas as pd
import numpy as np
import sys
!{sys.executable} -m pip install pandas-profiling

import re, string, unicodedata
import contractions
import inflect
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer, SnowballStemmer

from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, HashingVectorizer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.svm import SVC
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score


from sklearn.base import BaseEstimator, ClassifierMixin

import matplotlib.pyplot as plt

You should consider upgrading via the 'c:\Users\Julian\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


## Perfilamiento y entendimiento de datos

In [4]:
import pandas as pd

df = pd.read_csv("fake_news_spanish.csv", sep=";",encoding="utf-8" )
df_t=df

In [5]:
df_t 

,ID,Label,Titulo,Descripcion,Fecha
0,ID,1,'The Guardian' va con Sánchez: 'Europa necesit...,El diario británico publicó este pasado jueves...,02/06/2023
1,ID,0,REVELAN QUE EL GOBIERNO NEGOCIO LA LIBERACIÓN ...,REVELAN QUE EL GOBIERNO NEGOCIO LA LIBERACIÓN ...,01/10/2023
2,ID,1,El 'Ahora o nunca' de Joan Fuster sobre el est...,El valencianismo convoca en Castelló su fiesta...,25/04/2022
3,ID,1,"Iglesias alienta a Yolanda Díaz, ERC y EH Bild...","En política, igual que hay que negociar con lo...",03/01/2022
4,ID,0,Puigdemont: 'No sería ninguna tragedia una rep...,"En una entrevista en El Punt Avui, el líder de...",09/03/2018
...,...,...,...,...,...
57058,ID,1,El Defensor del Pueblo reclama a la Comunidad ...,El gobierno regional han indicado que la atenc...,08/06/2021
57059,ID,0,El EQUO plantea ceder la presidencia de la Com...,Si la higiene democrática nos lleva a esa exig...,08/09/2020
57060,ID,1,Alberto Garzón: 'Que los Borbones son unos lad...,El coordinador federal de IU asegura que la mo...,12/07/2018
57061,ID,1,Vox exige entrar en el Gobierno de Castilla y ...,Santiago Abascal: Vox tiene el derecho y el de...,13/02/2022


In [6]:
df_t.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57063 entries, 0 to 57062
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   ID           57063 non-null  object
 1   Label        57063 non-null  int64 
 2   Titulo       57047 non-null  object
 3   Descripcion  57063 non-null  object
 4   Fecha        57063 non-null  object
dtypes: int64(1), object(4)
memory usage: 2.2+ MB


## Entendimiento de de los datos

In [7]:
import pandas as pd
from scipy import stats as st

# Copia del dataset original para evitar modificarlo directamente
textos = df_t.copy()

# Longitud total del mensaje en caracteres
textos['Conteo'] = textos['Descripcion'].apply(len)

# Longitud de la palabra más larga en cada mensaje
textos['Max'] = textos['Descripcion'].apply(lambda x: max(map(len, x.split())))

# Longitud de la palabra más corta en cada mensaje
textos['Min'] = textos['Descripcion'].apply(lambda x: min(map(len, x.split())))

# Moda de la longitud de palabras en cada mensaje
#textos['Moda'] = textos['Descripcion'].apply(lambda x: st.mode([len(p) for p in x.split()])[0][0])

# Mostrar resultados
print(textos.head())

   ID  Label                                             Titulo  \
0  ID      1  'The Guardian' va con Sánchez: 'Europa necesit...   
1  ID      0  REVELAN QUE EL GOBIERNO NEGOCIO LA LIBERACIÓN ...   
2  ID      1  El 'Ahora o nunca' de Joan Fuster sobre el est...   
3  ID      1  Iglesias alienta a Yolanda Díaz, ERC y EH Bild...   
4  ID      0  Puigdemont: 'No sería ninguna tragedia una rep...   

                                         Descripcion       Fecha  Conteo  Max  \
0  El diario británico publicó este pasado jueves...  02/06/2023     218   13   
1  REVELAN QUE EL GOBIERNO NEGOCIO LA LIBERACIÓN ...  01/10/2023    3251   14   
2  El valencianismo convoca en Castelló su fiesta...  25/04/2022     169   13   
3  En política, igual que hay que negociar con lo...  03/01/2022     133   15   
4  En una entrevista en El Punt Avui, el líder de...  09/03/2018     242   14   

   Min  
0    1  
1    1  
2    2  
3    2  
4    1  


In [8]:
textos

,ID,Label,Titulo,Descripcion,Fecha,Conteo,Max,Min
0,ID,1,'The Guardian' va con Sánchez: 'Europa necesit...,El diario británico publicó este pasado jueves...,02/06/2023,218,13,1
1,ID,0,REVELAN QUE EL GOBIERNO NEGOCIO LA LIBERACIÓN ...,REVELAN QUE EL GOBIERNO NEGOCIO LA LIBERACIÓN ...,01/10/2023,3251,14,1
2,ID,1,El 'Ahora o nunca' de Joan Fuster sobre el est...,El valencianismo convoca en Castelló su fiesta...,25/04/2022,169,13,2
3,ID,1,"Iglesias alienta a Yolanda Díaz, ERC y EH Bild...","En política, igual que hay que negociar con lo...",03/01/2022,133,15,2
4,ID,0,Puigdemont: 'No sería ninguna tragedia una rep...,"En una entrevista en El Punt Avui, el líder de...",09/03/2018,242,14,1
...,...,...,...,...,...,...,...,...
57058,ID,1,El Defensor del Pueblo reclama a la Comunidad ...,El gobierno regional han indicado que la atenc...,08/06/2021,244,12,1
57059,ID,0,El EQUO plantea ceder la presidencia de la Com...,Si la higiene democrática nos lleva a esa exig...,08/09/2020,314,11,1
57060,ID,1,Alberto Garzón: 'Que los Borbones son unos lad...,El coordinador federal de IU asegura que la mo...,12/07/2018,70,11,2
57061,ID,1,Vox exige entrar en el Gobierno de Castilla y ...,Santiago Abascal: Vox tiene el derecho y el de...,13/02/2022,232,12,1


## Preparación de los datos
Para poder realizar el pre-procesamiento de los datos, se debe pasar por:
- Limpieza de los datos
- Tokenización de los datos
- normalización de los datos

### Limpieza de los datos

Se debe dejar el archivo csv en un texto plano, también se elimina caracteres especiales y se pasa todo a minúsculas

In [9]:
def remove_non_ascii(words):
    new_words = []
    for word in words:
        if word is not None:
            new_word = unicodedata.normalize('NFKD', word).encode('ascii', 'ignore').decode('utf-8', 'ignore')
            new_words.append(new_word)
    return new_words

def to_lowercase(words):
    new_words = []
    for word in words:
        if word is not None:
            new_word = word.lower()
            new_words.append(new_word)
    return new_words

def replace_numbers(words):
    p = inflect.engine()
    new_words = []
    for word in words:
        if word.isdigit():
            new_word = p.number_to_words(word)
            new_words.append(new_word)
        else:
            new_words.append(word)
    return new_words

def remove_punctuation(words):
    new_words = []
    for word in words:
        if word is not None:
            new_word = re.sub(r'[^\w\s]', '', word)
            if new_word != '':
                new_words.append(new_word)
    return new_words

def remove_stopwords(words):
    new_words = []
    for word in words:
        if word not in stopwords.words('spanish'):
            new_words.append(word)
    return new_words

def processing(words):
    words = remove_non_ascii(words)
    words = to_lowercase(words)
    words = replace_numbers(words)
    words = remove_punctuation(words)
    words = remove_stopwords(words)
    return words

## Tokenización

La tokenización es el proceso de dividir un texto en palabras individuales. Pero antes se realiza una correción de las contraciones que pueden aparecer en el texto.

In [10]:
df_t['Descripcion'] = df_t['Descripcion'].apply(contractions.fix)

In [11]:
nltk.download('punkt_tab')
df_t['words'] = df_t['Descripcion'].apply(word_tokenize)
df_t.head()

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Julian\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


,ID,Label,Titulo,Descripcion,Fecha,words
0,ID,1,'The Guardian' va con Sánchez: 'Europa necesit...,El diario británico publicó este pasado jueves...,02/06/2023,"[El, diario, británico, publicó, este, pasado,..."
1,ID,0,REVELAN QUE EL GOBIERNO NEGOCIO LA LIBERACIÓN ...,REVELAN QUE EL GOBIERNO NEGOCIO LA LIBERACIÓN ...,01/10/2023,"[REVELAN, QUE, EL, GOBIERNO, NEGOCIO, LA, LIBE..."
2,ID,1,El 'Ahora o nunca' de Joan Fuster sobre el est...,El valencianismo convoca en Castelló su fiesta...,25/04/2022,"[El, valencianismo, convoca, en, Castelló, su,..."
3,ID,1,"Iglesias alienta a Yolanda Díaz, ERC y EH Bild...","En política, igual que hay que negociar con lo...",03/01/2022,"[En, política, ,, igual, que, hay, que, negoci..."
4,ID,0,Puigdemont: 'No sería ninguna tragedia una rep...,"En una entrevista en El Punt Avui, el líder de...",09/03/2018,"[En, una, entrevista, en, El, Punt, Avui, ,, e..."


In [12]:
df_t['words'].dropna()

0        [El, diario, británico, publicó, este, pasado,...
1        [REVELAN, QUE, EL, GOBIERNO, NEGOCIO, LA, LIBE...
2        [El, valencianismo, convoca, en, Castelló, su,...
3        [En, política, ,, igual, que, hay, que, negoci...
4        [En, una, entrevista, en, El, Punt, Avui, ,, e...
                               ...                        
57058    [El, gobierno, regional, han, indicado, que, l...
57059    [Si, la, higiene, democrática, nos, lleva, a, ...
57060    [El, coordinador, federal, de, IU, asegura, qu...
57061    [Santiago, Abascal, :, Vox, tiene, el, derecho...
57062    [Los, Mossos, dEsquadra, han, blindado, los, a...
Name: words, Length: 57063, dtype: object

In [13]:
df_t.shape

(57063, 6)

In [16]:
df_t['words'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 57063 entries, 0 to 57062
Series name: words
Non-Null Count  Dtype 
--------------  ----- 
57063 non-null  object
dtypes: object(1)
memory usage: 445.9+ KB


In [17]:
#aplica la limpieza de datos hecho anteriormente
df_t['words1']= df_t['words'].apply(processing)
df_t.head()

,ID,Label,Titulo,Descripcion,Fecha,words,words1
0,ID,1,'The Guardian' va con Sánchez: 'Europa necesit...,El diario británico publicó este pasado jueves...,02/06/2023,"[El, diario, británico, publicó, este, pasado,...","[diario, britanico, publico, pasado, jueves, e..."
1,ID,0,REVELAN QUE EL GOBIERNO NEGOCIO LA LIBERACIÓN ...,REVELAN QUE EL GOBIERNO NEGOCIO LA LIBERACIÓN ...,01/10/2023,"[REVELAN, QUE, EL, GOBIERNO, NEGOCIO, LA, LIBE...","[revelan, gobierno, negocio, liberacion, mirel..."
2,ID,1,El 'Ahora o nunca' de Joan Fuster sobre el est...,El valencianismo convoca en Castelló su fiesta...,25/04/2022,"[El, valencianismo, convoca, en, Castelló, su,...","[valencianismo, convoca, castello, fiesta, gra..."
3,ID,1,"Iglesias alienta a Yolanda Díaz, ERC y EH Bild...","En política, igual que hay que negociar con lo...",03/01/2022,"[En, política, ,, igual, que, hay, que, negoci...","[politica, igual, negociar, empresarios, negoc..."
4,ID,0,Puigdemont: 'No sería ninguna tragedia una rep...,"En una entrevista en El Punt Avui, el líder de...",09/03/2018,"[En, una, entrevista, en, El, Punt, Avui, ,, e...","[entrevista, punt, avui, lider, jxcat, desdram..."


## Normalización 
 En esta etapa se elimina los prefijos y los sufijos de las palabras, para que las palabras se puedan comparar de manera más efectiva.

In [18]:
import stanza
from nltk.stem import SnowballStemmer
from tqdm import tqdm
import pandas as pd
import json
import os

# Descargar el modelo de Stanza para español (solo la primera vez)
stanza.download("es")

# Inicializar el pipeline de Stanza
nlp_stanza = stanza.Pipeline("es", processors="tokenize,mwt,pos,lemma")

# Inicializar el stemmer de NLTK
stemmer = SnowballStemmer("spanish")

# Función para aplicar stemming a una lista de palabras
def stem_words(words):
    return [stemmer.stem(word) for word in words]

# Función para lematizar una lista de textos
def lemmatize_text(texts):
    lemmatized_texts = []
    for text in tqdm(texts, desc="Lematizando textos"):
        doc = nlp_stanza(text)
        lemmatized_texts.append([word.lemma for sentence in doc.sentences for word in sentence.words])
    return lemmatized_texts

# Función para aplicar stemming y lematización a una lista de listas de palabras
def stem_and_lemmatize(words_list):
    texts = [" ".join(words) for words in words_list]  # Convertir listas de palabras en texto
    stems = [stem_words(words) for words in tqdm(words_list, desc="Aplicando stemming")]
    lemmas = lemmatize_text(texts)  # Lematización
    return [{"stems": stem, "lemas": lema} for stem, lema in zip(stems, lemmas)]

# Nombre del archivo donde se guardarán o leerán los resultados
output_filename = "processed_results.json"

# Verificar si el archivo existe y no está vacío
if os.path.exists(output_filename) and os.path.getsize(output_filename) > 0:
    print("Cargando resultados preprocesados desde el archivo...")
    with open(output_filename, 'r') as f:
        df_t['words2'] = json.load(f)
else:
    print("Procesando datos (stemming y lematización)...")
    df_t['words2'] = stem_and_lemmatize(df_t['words1'].tolist())
    # Guardar los resultados en el archivo JSON
    with open(output_filename, 'w') as f:
        json.dump(df_t['words2'].tolist(), f)
    print("Resultados guardados en:", output_filename)

# Mostrar las primeras filas del DataFrame
print(df_t.head())

c:\Users\Julian\AppData\Local\Programs\Python\Python39\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-02-21 14:34:04 INFO: Downloaded file to C:\Users\Julian\stanza_resources\resources.json
2025-02-21 14:34:04 INFO: Downloading default packages for language: es (Spanish) ...
2025-02-21 14:34:06 INFO: File exists: C:\Users\Julian\stanza_resources\es\default.zip
2025-02-21 14:34:09 INFO: Finished downloading models and saved to C:\Users\Julian\stanza_resources
2025-02-21 14:34:09 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES
2025-02-21 14:34:09 INFO: Downloaded file to C:\Users\Julian\stanza_resources\resources.json
2025-02-21 14:34:09 INFO: Loading these models fo

Cargando resultados preprocesados desde el archivo...
   ID  Label                                             Titulo  \
0  ID      1  'The Guardian' va con Sánchez: 'Europa necesit...   
1  ID      0  REVELAN QUE EL GOBIERNO NEGOCIO LA LIBERACIÓN ...   
2  ID      1  El 'Ahora o nunca' de Joan Fuster sobre el est...   
3  ID      1  Iglesias alienta a Yolanda Díaz, ERC y EH Bild...   
4  ID      0  Puigdemont: 'No sería ninguna tragedia una rep...   

                                         Descripcion       Fecha  \
0  El diario británico publicó este pasado jueves...  02/06/2023   
1  REVELAN QUE EL GOBIERNO NEGOCIO LA LIBERACIÓN ...  01/10/2023   
2  El valencianismo convoca en Castelló su fiesta...  25/04/2022   
3  En política, igual que hay que negociar con lo...  03/01/2022   
4  En una entrevista en El Punt Avui, el líder de...  09/03/2018   

                                               words  \
0  [El, diario, británico, publicó, este, pasado,...   
1  [REVELAN, QUE, EL, 

### Selección de campos
Primero, se prepara la variable predictora que en este caso es ..., y los textos que se van a utilizar.

In [19]:
df_t['words2'] = df_t['words2'].apply(lambda x: ' '.join(map(str, x)))
df_t

,ID,Label,Titulo,Descripcion,Fecha,words,words1,words2
0,ID,1,'The Guardian' va con Sánchez: 'Europa necesit...,El diario británico publicó este pasado jueves...,02/06/2023,"[El, diario, británico, publicó, este, pasado,...","[diario, britanico, publico, pasado, jueves, e...",stems lemas
1,ID,0,REVELAN QUE EL GOBIERNO NEGOCIO LA LIBERACIÓN ...,REVELAN QUE EL GOBIERNO NEGOCIO LA LIBERACIÓN ...,01/10/2023,"[REVELAN, QUE, EL, GOBIERNO, NEGOCIO, LA, LIBE...","[revelan, gobierno, negocio, liberacion, mirel...",stems lemas
2,ID,1,El 'Ahora o nunca' de Joan Fuster sobre el est...,El valencianismo convoca en Castelló su fiesta...,25/04/2022,"[El, valencianismo, convoca, en, Castelló, su,...","[valencianismo, convoca, castello, fiesta, gra...",stems lemas
3,ID,1,"Iglesias alienta a Yolanda Díaz, ERC y EH Bild...","En política, igual que hay que negociar con lo...",03/01/2022,"[En, política, ,, igual, que, hay, que, negoci...","[politica, igual, negociar, empresarios, negoc...",stems lemas
4,ID,0,Puigdemont: 'No sería ninguna tragedia una rep...,"En una entrevista en El Punt Avui, el líder de...",09/03/2018,"[En, una, entrevista, en, El, Punt, Avui, ,, e...","[entrevista, punt, avui, lider, jxcat, desdram...",stems lemas
...,...,...,...,...,...,...,...,...
57058,ID,1,El Defensor del Pueblo reclama a la Comunidad ...,El gobierno regional han indicado que la atenc...,08/06/2021,"[El, gobierno, regional, han, indicado, que, l...","[gobierno, regional, indicado, atencion, dia, ...",stems lemas
57059,ID,0,El EQUO plantea ceder la presidencia de la Com...,Si la higiene democrática nos lleva a esa exig...,08/09/2020,"[Si, la, higiene, democrática, nos, lleva, a, ...","[si, higiene, democratica, lleva, exigencia, t...",stems lemas
57060,ID,1,Alberto Garzón: 'Que los Borbones son unos lad...,El coordinador federal de IU asegura que la mo...,12/07/2018,"[El, coordinador, federal, de, IU, asegura, qu...","[coordinador, federal, iu, asegura, monarquia,...",stems lemas
57061,ID,1,Vox exige entrar en el Gobierno de Castilla y ...,Santiago Abascal: Vox tiene el derecho y el de...,13/02/2022,"[Santiago, Abascal, :, Vox, tiene, el, derecho...","[santiago, abascal, vox, derecho, deber, forma...",stems lemas


In [20]:
X_data, y_data = df_t['words2'], df_t['Label']

y_data = (y_data == 1).astype(int)


print(y_data.value_counts())

Label
1    33158
0    23905
Name: count, dtype: int64


Se transforma el texto con una transformación binaria 1 a 1

In [21]:
dummy = CountVectorizer(binary=True)
X_dummy = dummy.fit_transform(X_data)
print(X_dummy.shape)
X_dummy.toarray()[0]

(57063, 2)


array([1, 1])

## Creación y aplicación de los modelos

## TF-IDF + XGBoost

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords


vectorizer = TfidfVectorizer(max_features=50000, ngram_range=(1, 2))

X_tfidf = vectorizer.fit_transform(X_data)
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y_data, test_size=0.2, random_state=42)

model = XGBClassifier(n_estimators=50, learning_rate=0.05, scale_pos_weight=0.5, random_state=42)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)[:,1]
threshold = 0.7
y_pred = (y_pred >= threshold).astype(int)

precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)

print(f'Accuracy: {accuracy:.2f}')
print(f"Precisión: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1-Score: {f1:.2f}")